This script computes the 'baseline' inter-subject correlations of neural activity during video watching.
'Baseline' is meant in the sense of 'not associated with predictors such as ideology', simply shared activity across all subjects regardless of creed etc.
This is the first step in many ISC papers.

This script requires processed data from the .py scripts in this folder. Use the .sh scripts to send the baseline_ISC_subject.py jobs to your cluster. Then use submit_baseline_ISC_group_permutation_jobs.sh to run the permutation tests. Then use this notebook to load results & apply FDR correction.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys, nltools, scipy, glob
from nltools import Brain_Data, Design_Matrix, expand_mask, collapse_mask
from nltools.plotting import plot_brain, plot_glass_brain
from nilearn.plotting import plot_stat_map, plot_img, plot_roi
import nibabel as nib

from helpers import load_cleaned_funx

## Choose run

In [ ]:
run = 2 # == video number (1 = BBC Earth, 2 = PBS News, 3 = CNN Debate)

## Settings

In [ ]:
# Settings
base_dir = 'BASE_DIR_HERE/'
mask_dir = 'MASK_DIR_HERE'
mask_fname = (mask_dir + '/80pct_brain_mask.nii.gz') # This is a mask covering nearly all of the brain: where 80% or more of subjects had measurements (i.e. excluding part of cerebellum)
all_subs = pd.read_csv(base_dir + 'Data/Subjects_and_exclusions/all_subjects.csv')['sub'].values.tolist()
print(len(all_subs))

# Exclusions
exclude_motion = pd.read_csv(base_dir + '/Data/Subjects_and_exclusions/exclude_video-watching_motion.csv'
                            ).query('run == @run')['sub'].values.tolist()
exclude_attention = pd.read_csv(base_dir + '/Data/Subjects_and_exclusions/exclude_video-watching_attention.csv'
                            ).query('run == @run')['sub'].values.tolist()
exclude = exclude_motion + exclude_attention
if run == 1:
    last_TR = 390
elif run == 2:
    last_TR = 307
elif run == 3:
    last_TR = 720
print('Exclusions: %s'%exclude)
sub_list = [i for i in all_subs if i not in exclude]
print('%i subjects remain: %s'%(len(sub_list),sub_list))

## Load permutation test results per voxel, append, plot

In [ ]:
alldat = pd.DataFrame()
for voxgr in range(73):
    if np.mod(voxgr,10)==0:
        print(voxgr, end = ',')
    dat = pd.read_csv(base_dir + 'Results/Inter-subject_correlations/baseline_ISC/'+
                                 'run-%i_r_permutation-test_voxgr-%i.csv'%(run,voxgr), index_col = 0)
    alldat = alldat.append(dat)
alldat = alldat.reset_index(drop=True)

In [ ]:
alldat.shape

In [ ]:
r_map = Brain_Data(mask_fname, mask = mask_fname)
r_map.data = alldat['r'].values

In [ ]:
plot_stat_map(r_map.to_nifti())

In [ ]:
p_map = Brain_Data(mask_fname, mask = mask_fname)
p_map.data = alldat['p'].values

In [ ]:
r_map.write(base_dir + 'Results/Inter-subject_correlations/baseline_ISC/'+
                                 'baseline_ISC_run-%i_r.nii.gz'%(run))
p_map.write(base_dir + 'Results/Inter-subject_correlations/baseline_ISC/'+
                                 'baseline_ISC_run-%i_p.nii.gz'%(run))

##### Threshold using FDR

In [ ]:
ps = alldat['p'].values
qs = [0.001, 0.05]
for q in qs:
    p_thresh_fdr = nltools.stats.fdr(ps, q = q)
    print('q = %.4f, p_corr = %.6f'%(q,p_thresh_fdr))
    p_thresh_arr = p_map.data <= p_thresh_fdr
    r_map_thresh_fdr = r_map.copy()
    r_map_thresh_fdr.data = np.multiply(r_map_thresh_fdr.data,p_thresh_arr)
    plot_stat_map(r_map_thresh_fdr.to_nifti(), vmax = .6)
    plt.show()
    print('Writing...')
    r_map_thresh_fdr.write(base_dir + 'Results/Inter-subject_correlations/baseline_ISC/'+
                                      'baseline_ISC_run-%i_r_thresh-p-fdr-%.3f.nii.gz'%(run,q))